In [3]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict,Counter
from random import sample

In [10]:
class NeuralNet:
    def __init__(self,nodes,lamb=0.0,alpha=0.1,eps=0.0):
        '''
        Constructor for neural net
        nodes - list detailing number of nodes in each layer
        lamb - regularization
        alpha - learning rate
        eps - cost function stopping condition
        '''
        self.nodes = nodes
        self.lamb = lamb
        self.alpha = alpha
        self.weights = []
        self.eps = eps
        #initialize weights for each layer, include bias
        for i in range(len(nodes)-1):
            self.weights.append(np.random.normal(0,1,(nodes[i]+1,nodes[i+1])))
    
    def get_sigmoid(self, x):
        return 1 / (1+np.exp(-x))
    
    def deriv_sigmoid(self, x):
        return x * (1-x)

    def train(self, features, targs):
        prev_cost = -math.inf
        total_delta = [0]*len(self.weights)
        num_inst = len(targs)
        keep_learn = True

        while(keep_learn):
            J = 0
            for instance,target in zip(features,targs):
                #iterate through layers, vectorize forward pass
                activations = [np.atleast_2d(instance)]
                for i in range(len(self.weights)-1):
                    this_a = self.get_sigmoid(self.weights[i].dot(activations[i].T))
                    activations.append(np.insert(this_a,0,1))
                activations.append(self.get_sigmoid(activations[len(self.weights)-1].dot(self.weights[len(self.weights)-1])))
                guess = activations[-1]

                #accumulate sum loss
                J += np.sum((-target).dot(math.log(guess)) - (1-target).dot(math.log(1-guess)))

                #begin backwards propogation
                error = guess - target
                delta_inst = [error]

                #get delta values for all weights on current instance
                for i in range(len(self.weights)-1, 0, -1):
                    #this_del = (self.weights[i].T.dot(delta_inst[-1])) * self.deriv_sigmoid(activations[i])
                    this_del = (self.weights[i].T*(delta_inst[-1])) * self.deriv_sigmoid(activations[i])
                    delta_inst.append(this_del[0][1:])

                #reverse delta values
                delta_inst = delta_inst[::-1]

                #accumulate gradients
                for i in range(len(self.weights)-1,-1,-1):
                    total_delta[i] += (delta_inst[i]*(activations[i].T)).T
            
            #regularize weights and update
            for i in range(len(self.weights)-1,-1,-1):
                P = self.lamb * (self.weights[i]).T
                total_delta[i] = total_delta[i] + P
                total_delta[i] = total_delta[i] / num_inst
                learn_diff = self.alpha * total_delta[i]
                self.weights[i] = self.weights[i] - learn_diff.T

            J /= num_inst
            curr_s = 0
            for i in range(len(self.weights)):
                curr_s += np.sum(self.weights[i][1:]**2)

            #curr_s = np.sum(self.weights[1:]**2)
            curr_s *= (self.lamb/(2*num_inst))
            new_cost = J + curr_s

            #if improvement in cost is less than epsilon, stop
            if new_cost - prev_cost < self.eps:
                keep_learn = False

            prev_cost = new_cost

    #def calculate_loss(self,data,targets):
        #predictions = aaaa

    def predict(self,instance):
        pred = [np.ones(len(instance)),instance]

        for i in len(self.weights):
            pred = self.sigmoid(np.dot(pred,self.weights[i]))
        
        return pred

'''
TODO: 
    - Implement calculate_loss
    - Debug using sample data
    - Swag, nae nae, and finesse
'''



'\nTODO: \n    - Implement calculate_loss\n    - Debug using sample data\n    - Swag, nae nae, and finesse\n'

In [11]:
'''
Training set
	Training instance 1
		x: [0.13000]
		y: [0.90000]
	Training instance 2
		x: [0.42000]
		y: [0.23000]
        Initial Theta1 (the weights of each neuron, including the bias weight, are stored in the rows):
	0.40000  0.10000  
	0.30000  0.20000  

Initial Theta2 (the weights of each neuron, including the bias weight, are stored in the rows):
	0.70000  0.50000  0.60000  
'''
np.array([[0.40000,0.10000 ],[0.30000,0.20000 ]])
np.array([[0.7],[0.5],[0.6]])
train_set_1 = {'x': [0.13000,0.42000], 'y': [0.90000,0.23000]}
X = np.array([[1,0.13000],[1,0.42000]])
Y = np.array([[0.90000],[0.23000]])
train_df = pd.DataFrame(data=train_set_1)
train_df.insert(0,'bias',np.ones)
train_nn = NeuralNet([1,2,1],eps=0.001)
train_nn.train(X,Y)



In [12]:
#implement backprogagation algorithm in a way that allows you to specify how many layers and neurons you would like your neural network to have
#https://pyimagesearch.com/2021/05/06/backpropagation-from-scratch-with-python/
test = [1,2,3,4,5]
out = []
for i in range(len(test),0,-1):
    out.append(i)
#print(':3')
print(out)

[5, 4, 3, 2, 1]
